# CONSUMER

Creazione consumer kafka.

Il consumer scarica i dati dal topic precedentemente creato e li carica nell'appropriata collection MongoDB

In [1]:
### CONSUMER
from kafka import KafkaConsumer
import time
import ast

# creazione consumer
consumer = KafkaConsumer(bootstrap_servers='sandbox-hdp.hortonworks.com:6667',
    auto_offset_reset='earliest',
    consumer_timeout_ms=1000)
consumer.subscribe(['f1_germania'])

In [2692]:
# riempimento DB con dati raccolti
import pymongo
from pymongo import MongoClient

# creazione db
client = MongoClient('localhost', 27017)
db = client.germania
# creazione collection
official = db.official
all_tweet = db.all_tweet
tweet_x_piloti = db.tweet_x_piloti
tweet_piloti = db.tweet_piloti
tweet_scuderie = db.tweet_scuderie

# user id piloti
piloti = ['213969309',         
          '1143472657',        
          '262230432',         
          '556260847',         
          '537951506',         
          '850708922054959104',
          '971608718',         
          '93819292',          
          '214413743',         
          '516464106',         
          '353786894',         
          '397306167',         
          '217461138',         
          '68674747',          
          '2222976084',        
          '78502161',          
          '4201552600',        
          '394892872',         
          '1386735721']

# hashtag dei piloti
hashtag_piloti = ['#LH44', '#TeamLH', '#Hamilton',
                  '#VB77', '#Bottas',
                  '#Vettel', '#Seb5', '#SebastianVettel',
                  '#CL16', '#Leclerc', '#Charles16',
                  '#Verstappen', '#MV33', '#gomax',
                  '#GAS10', '#Gasly', '#PG10',
                  '#Kimi7', '#Raikkonen', '#KR7',
                  '#Giovinazzi', '#AG99',
                  '#NH27', '#Hulk27', '#Hulkenberg',
                  '#DR3', '#Ricciardo',
                  '#L4ando', '#Norris', '#LN4',
                  '#CS55', '#Sainz', '#Carlo55ainz',
                  '#AA23', '#Albon',
                  '#DK26', '#Kvyat',
                  '#R8G', '#Grosjean',
                  '#KM20', '#Magnussen',
                  '#Checo11', '#Perez',
                  '#Stroll',
                  '#GR63', '#Russell',
                  '#RK88', '#Kubica']

# user id delle scuderie
scuderie = ['26235265',  
            '108247668', 
            '226087776', 
            '229454266', 
            '28297965',  
            '24871896',  
            '2303406624',
            '16143542',  
            '90636188',  
            '82322622']  

# caricamento dei tweet all'interno del db
for message in consumer:
    message = message.value.decode("utf-8")
    message = ast.literal_eval(message)
    
    # tweet canale uffciale
    if message['user']['id'] == '69008563':
        official.insert_one(message)
    # tweet creati dai piloti
    if message['user']['id'] in piloti:
        tweet_piloti.insert_one(message)
    # tweet riguardanti i piloti
    for i in hashtag_piloti:
        if i in message['text']:
            tweet_x_piloti.insert_one(message)
    # tweet creati dalle scuderie
    if message['user']['id'] in scuderie:
        tweet_scuderie.insert_one(message)
    # tutti i tweet
    all_tweet.insert_one(message)
